Training Using Google Colab

In [ ]:
# ✅ Step 1: Install dependencies
!pip install -q transformers datasets accelerate bitsandbytes trl peft

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-3.2-1B"

# The instruction dataset to use
dataset_name = "/content/medical_dataset_llama3_format.json"

# Fine-tuned model name
new_model = "chat-finetune"

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
from datasets import load_dataset

# Load dataset from local JSON file
dataset = load_dataset("json", data_files="/content/medical_dataset_llama3_format.json")

# Perform train-test split (80% train, 20% test)
dataset = dataset["train"].train_test_split(test_size=0.2)

# Print sample entry from the train set
print(dataset["train"][0])


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.to(device)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
def tokenize_function(examples):
    texts = []
    for msgs in examples["messages"]:
        text = " ".join([msg["content"] for msg in msgs if "content" in msg])  # Extract and concatenate text
        texts.append(text)
    
    return tokenizer(texts, truncation=True, max_length=512, padding="max_length")

dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)


training_arguments = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps=100,
    logging_steps=50,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=False,
    max_grad_norm=1.0,
    max_steps=500,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    args=training_arguments,
)

In [ ]:
trainer.train()

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
#Testing the model
# Ignore warnings
import logging
logging.getLogger("transformers").setLevel(logging.CRITICAL)

from transformers import pipeline

# Run text generation pipeline with our next model
prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat is a cause of heart attack?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)

print(result[0]['generated_text'])